In [ ]:
import hopsworks
import pandas as pd
import json

from confluent_kafka import Producer

In [ ]:
project = hopsworks.login()

In [ ]:
pdf = pd.read_csv("hdfs:///Projects/live_coding/RawData/transactions.csv")

In [ ]:
pdf.head(10)

In [ ]:
pdf = pdf[['tid', 'datetime', 'amount', 'cc_num']]

In [ ]:
pdf['datetime'] = pd.Timestamp.now() # Simulate incoming transactions 

In [ ]:
pdf['datetime'] = pdf.datetime.map(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
kafka_api = project.get_kafka_api()

In [ ]:
config = kafka_api.get_default_config()

In [ ]:
config

In [ ]:
producer = Producer(config)

In [ ]:
data = json.loads(pdf.to_json(orient="records"))

In [ ]:
data[:5]

In [ ]:
batch_size = 0
for t in data:
    producer.produce("live_card_transactions", json.dumps(t))
    batch_size += 1
    
    if batch_size == 100:
        producer.flush()
        batch_size = 0